<a href="https://colab.research.google.com/github/ShubhangiSRG24/ML_codesamples/blob/main/Recommendation_System_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   876k      0  0:00:01  0:00:01 --:--:--  877k


In [8]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [11]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [12]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [13]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [15]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [18]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [19]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [20]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0477, 0.0334, 0.0148,  ..., 0.0019, 0.0214, 0.0110],
        [0.0266, 0.0230, 0.0132,  ..., 0.0395, 0.0476, 0.0165],
        [0.0472, 0.0350, 0.0338,  ..., 0.0421, 0.0235, 0.0141],
        ...,
        [0.0405, 0.0383, 0.0363,  ..., 0.0465, 0.0096, 0.0454],
        [0.0292, 0.0262, 0.0493,  ..., 0.0470, 0.0293, 0.0303],
        [0.0005, 0.0214, 0.0470,  ..., 0.0296, 0.0451, 0.0213]])
item_factors.weight tensor([[0.0173, 0.0451, 0.0416,  ..., 0.0495, 0.0457, 0.0442],
        [0.0224, 0.0037, 0.0105,  ..., 0.0327, 0.0094, 0.0003],
        [0.0318, 0.0398, 0.0136,  ..., 0.0435, 0.0076, 0.0450],
        ...,
        [0.0334, 0.0452, 0.0049,  ..., 0.0029, 0.0213, 0.0053],
        [0.0053, 0.0345, 0.0427,  ..., 0.0358, 0.0323, 0.0054],
        [0.0246, 0.0135, 0.0256,  ..., 0.0454, 0.0481, 0.0047]])


In [21]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-21-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.074452146660859
iter #1 Loss: 4.741531760559469
iter #2 Loss: 2.4754071433834617
iter #3 Loss: 1.7199800907052714
iter #4 Loss: 1.344712945637364
iter #5 Loss: 1.127702804537594
iter #6 Loss: 0.990371747140957
iter #7 Loss: 0.899447129992059
iter #8 Loss: 0.836697771161946
iter #9 Loss: 0.7914439063084308
iter #10 Loss: 0.7590658654870116
iter #11 Loss: 0.7342581953569717
iter #12 Loss: 0.7159548228616037
iter #13 Loss: 0.7013787340542992
iter #14 Loss: 0.6901364102581431
iter #15 Loss: 0.6816574055606943
iter #16 Loss: 0.6748662080029546
iter #17 Loss: 0.6698342671778601
iter #18 Loss: 0.6658629733445075
iter #19 Loss: 0.6627120480395211
iter #20 Loss: 0.6606294043778163
iter #21 Loss: 0.6591556769639707
iter #22 Loss: 0.6579692696935029
iter #23 Loss: 0.6565734648008636
iter #24 Loss: 0.6561124084262073
iter #25 Loss: 0.65503299675012
iter #26 Loss: 0.6544006017939693
iter #27 Loss: 0.6539441159533971
iter #28 Loss: 0.6528656954692705
iter #29 Loss: 0.651535806770857

In [22]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.2440,  1.2639,  1.5174,  ...,  0.8631,  1.3427,  0.5053],
        [ 0.2539,  0.6942,  1.1788,  ...,  2.1563,  1.5889,  0.6813],
        [ 1.3529,  0.2548,  2.4361,  ...,  0.9380, -2.4123,  2.4083],
        ...,
        [ 1.7127,  1.0033, -1.0321,  ...,  2.0789, -0.4138,  1.3333],
        [ 0.3978,  0.8810,  1.3914,  ...,  1.2671,  0.8235,  0.4102],
        [ 1.0060,  1.8650,  1.7304,  ...,  1.0620,  0.4197,  1.0520]],
       device='cuda:0')
item_factors.weight tensor([[ 0.7314,  0.4552,  0.6679,  ...,  0.3730,  0.5229,  0.6733],
        [-0.3567,  0.5415,  0.7834,  ...,  0.6894,  0.6795,  0.2080],
        [ 0.2105,  0.7536,  0.6923,  ...,  0.7322,  0.0682,  0.4689],
        ...,
        [ 0.3681,  0.3797,  0.3390,  ...,  0.3381,  0.3503,  0.3397],
        [ 0.3892,  0.4194,  0.4277,  ...,  0.4213,  0.4118,  0.3895],
        [ 0.4033,  0.3945,  0.4060,  ...,  0.4235,  0.4295,  0.3842]],
       device='cuda:0')


In [23]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [24]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [25]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [26]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Jurassic Park (1993)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 True Lies (1994)
	 Men in Black (a.k.a. MIB) (1997)
	 Mission: Impossible (1996)
	 Ace Ventura: Pet Detective (1994)
	 Mask, The (1994)
	 Die Hard (1988)
	 Die Hard: With a Vengeance (1995)
	 Groundhog Day (1993)
Cluster #1
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Lord of the Rings: The Return of the King, The (2003)
	 Back to the Future (1985)
	 Dark Knight, The (2008)
	 Princess Bride, The (1987)
Cluster #2
	 Pulp Fiction (1994)
	 Fight Club (1999)
	 Usual Suspects, The (1995)
	 American Beauty (1999)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Godfather, The (1972)
	 Fargo (1996)
	 Sixth Sense, The (1999)
	 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
	 Memento (2000)
Cluster #3
	 B